In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

import warnings
warnings.filterwarnings('ignore')

In [5]:
spark = SparkSession\
                .builder\
                .appName('DataFrame practice')\
                .getOrCreate()

df = spark.read.json('data.json')
df.show(5)

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [6]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



## 1. Which page did user_id "" not visit?

In [11]:
# filter for users with blank user id
blank_pages = df.filter(
    df.userId == ""
).select('page')\
    .alias('blank_pages')\
    .drop_duplicates()

# get all pages
all_pages = df.select('page').drop_duplicates()

for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

NextSong
Logout
Downgrade
Submit Upgrade
Upgrade
Submit Downgrade
Settings
Error
Save Settings


## 2. How many female users do we have

In [17]:
df.filter(
    df.gender == 'F'
).select('userId')\
    .drop_duplicates()\
    .count()

462

## 3. How many songs were played from the most played artist

In [28]:
df.filter(
    df.page == 'NextSong'
).select('Artist')\
    .groupBy('Artist')\
    .agg({'Artist': 'count'})\
    .withColumnRenamed('count(Artist)', "Artistcount")\
    .sort(desc('Artistcount'))\
    .show()

+--------------------+-----------+
|              Artist|Artistcount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
|       Justin Bieber|         43|
|      The Black Keys|         40|
|         OneRepublic|         37|
|        Jack Johnson|         36|
|                Muse|         36|
|           Radiohead|         31|
|        Taylor Swift|         29|
|Barry Tuckwell/Ac...|         28|
|               Train|         28|
|          Lily Allen|         28|
|          Nickelback|         27|
|           Metallica|         27|
|           Daft Punk|         27|
|          Kanye West|         26|
|          John Mayer|         24|
+--------------------+-----------+
only showing top 20 rows



In [29]:
spark.stop()